In [ ]:
from pavlov import *
monitor(-1)

In [ ]:
ax = stats.plot(['*grave-tap*', '*forked-moves', -1], 'elo-mohex', 'μ', ffill=True, skip=2)
ax.set_ylim(None, 0)

In [ ]:
from boardlaw.arena import plot
plot.heatmap('*forked-moves')

In [ ]:
import pandas as pd
from pavlov import storage
df = pd.DataFrame.from_dict(storage.snapshots(-3), orient='index')

In [ ]:
sd = storage.load_path(df.path.iloc[-1])

In [ ]:
import pickle
len(pickle.dumps(sd['agent']))/1e6

In [ ]:
runs.resolve(-3)

In [ ]:
stats.review('*same-gangs')

In [ ]:
stats.review(-3)